In [1]:
# -*- coding: utf-8 -*-
"""
@author: ChewingGumKing_OJF
"""

#loads necessary libraries
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import os
from datetime import datetime
from datetime import date
import openpyxl
from selenium.webdriver.common.action_chains import ActionChains
import os
import warnings
import json
import pandas as pd
import pyperclip as pc
from bs4 import BeautifulSoup as bs
import re
from random import randint
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

warnings.filterwarnings("ignore")


In [2]:
#driver=webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe')
from selenium import webdriver 



chrome_options = Options()

chrome_options.add_argument("--disable-extensions")
# linux only
# chrome_options.add_argument("--headless")
# chrome_options.headless = True 

driver = webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe',options=chrome_options)

In [3]:
url='https://www.intelligentinsurer.com/listing/events'

driver.get(url)
time.sleep(5)

soup=bs(driver.page_source,'lxml')
items=soup.find_all('div',{'class':'listing'})

len(items)

cards=list()
for m in items:
    ritle=' '.join(m.h2.text.split())
    try:
        link=m.find('a')['href']
    except:
        link=''
    try:
        info=' '.join(m.find_all('p')[-1].text.split())
    except:
        info=''
    da=' '.join(m.find('span',{'class':'date'}).text.split()).replace(' - ','/')
    
    lo=' '.join(m.find('span',{'class':'location'}).text.split())
    try:
        ven=' '.join(m.find('span',{'class':'venue'}).text.split())
    except:
        ven=''
    cards.append([ritle,link,info,da,lo,ven])

In [55]:
intelligent=[]
for s in cards:
    title=s[0].split('(')[0].strip()
    link=s[1]
    event_info=s[2]
    dat=s[3]
    if '/' in dat:
        st,en=da.split('/')
    else:
        st=en=da.strip()
    pick=[st,en]
    spl_dt_obj = [datetime.strptime(v, '%d-%m-%Y') for v in pick]
#         except:
#             spl_dt_obj = [datetime.strptime(v, '%d %b %Y') for v in pick]
    date_= [z.strftime('%Y-%m-%d') for z in spl_dt_obj]
    start_date=date_[0]
    end_date=date_[1]
    loc=s[4]
    ven=s[5]
    if link!='':
        driver.get(link)
        time.sleep(2)
        soupn=bs(driver.page_source,'lxml')
        try:
            sp=soupn.find('div',{'class':re.compile('atom-sectionMain rowHolder speakers.+')}).find('div',{'class':'row'}).find_all('div',{'class':re.compile('col-xs-12.+')})
        except:
            sp=soupn.find('div',{'class':'atom-sectionMain rowHolder speakers'}).find('div',{'class':'row'}).find_all('div',{'class':re.compile('col-xs-12.+')})
        speak=[]
        for j in sp:
            try:
                na=' '.join(j.find('div',{'class':'atom-fullname attendee-name'}).text.split())
            except:
                na=' '.join(j.find('div',{'class':'atom-fullname speaker-name'}).text.split())
            try:
                tle=' '.join(j.find('div',{'class':'atom-text1 attendee-title'}).text.split())
            except:
                tle=' '.join(j.find('div',{'class':'atom-text1 color speaker-title'}).text.split())
            ln='https://www.commercial-lines-innovation-europe.com'+j.find('a')['href']
            shu={
                'name':na,
                'title':tle,
                'link':ln
            }
            speak.append(shu)
        Speakerlist=json.dumps(speak,ensure_ascii=False)
    else:
        Speakerlist=''
    
    def gu(luc):
        google_url_for_location="https://www.google.com/search?q="+luc+"&oq="+luc+"&num=1"
        time.sleep(randint(0,3))
        driver.get(google_url_for_location)
        time.sleep(4)
        try:
            google_map_url=driver.find_element_by_id('lu_map').click()
        except:
            try:
                google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
            except:
                google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
        time.sleep(1)
        google_map_url=driver.current_url
#                 print(google_map_url)
        return(google_map_url)
        ######################################
    def get_google_map_url(location):
        try:
            return(gu(location))
        except:
            try:
                return(gu(location))
            except:
                sha=location.split(',')
                try:
                    gu(sha[-3])
                except:
                    try:
                        gu(sha[-2])
                    except:
                        try:
                            gu(sha[-1])
                        except Exception as e:
                            print(location, "; url didn't go through")
                            print(e)
                            return("")


    def countr(locale):
        try:
            google_url_for_location="https://www.google.com/search?q="+locale+"&oq="+locale+"&num=1"
            time.sleep(randint(0,3))
            driver.get(google_url_for_location)
            time.sleep(4)
            try:
                google_map_url=driver.find_element_by_id('lu_map').click()
            except:
                try:
                    google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
                except:
                    try:
                        google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
                    except:
                        google_url_for_location="https://www.google.com/search?q="+locale+"&oq="+locale+"&num=1"
                        time.sleep(randint(0,3))
                        driver.get(google_url_for_location)
                        time.sleep(4)
                        google_map_url=driver.find_element_by_id('lu_map').click()
            time.sleep(1)
            try:
                country=driver.find_element_by_class_name('x3AX1-LfntMc-header-title-VdSJob').text
            except:
                country=driver.find_element_by_class_name('bwoZTb').text
#                 print(country)
            return(country)
        except Exception as e:
            print(locale, ";country didn't go through")
            print(e)
            return("")

    org_name='Newton Media Ltd'
    org_web='https://www.intelligentinsurer.com/'
    org_pro='Intelligent Insurer is essential reading for buyers, capital providers and brokers of reinsurance and wholesale insurance. Our readers include senior executives at insurers, reinsurers, brokers and their advisors, who are responsible for driving the market forward through their innovation and experience.'
    logo=''    
    sponsor=''
    agendalist=''
    type_=''
    category=''
    time_=''
    ticket_list=''
    event_web=link
    mail_=json.dumps(['info@newtonmedia.co.uk'])
    if 'nline' in loc or 'ff Campus' in loc or loc=='' or loc==' ' or loc=='-' or loc=='Virtual' or 'nspecified' in loc or 'irtual' in loc or 'ebinar' in loc:
        on_off='1'
        city=''
        venue=''
        country=''
        googlePlaceUrl=''
    else:
        on_off='0'
        city=loc#.split()[-3]
        country=countr(city)
        if ven=='':
            venue=loc
        else:
            venue=ven+', '+loc
        googlePlaceUrl=get_google_map_url(venue)
    
    intelligent.append([link,title,start_date,end_date,time_,event_info,ticket_list,
                 org_pro,org_name,org_web,logo,sponsor,agendalist,
                 type_,category,city,country,venue,event_web,googlePlaceUrl,mail_,
                 Speakerlist,on_off])
    

In [56]:
len(intelligent)

3

In [57]:
intelligent

[['https://www.claims-innovation-usa.com/',
  'Claims Innovation USA',
  '2022-11-09',
  '2022-11-10',
  '',
  "Claims Innovation USA 2022 (July 11th-13th) Promises to bring you the cutting edge in claims intelligence with a carefully curated agenda designed to allow insurance carriers at all stages of maturity develop solutions to their most pressing claims challenges. Join over 1000 of your US claims colleagues and partners for 2 days of virtual content followed by an in-person gathering in the heart of Chicago. You'll hear business-critical insights on how to deliver a seamless customer experience, improve employee retention and advancement as well as driving further operational and cost efficiencies.",
  '',
  'Intelligent Insurer is essential reading for buyers, capital providers and brokers of reinsurance and wholesale insurance. Our readers include senior executives at insurers, reinsurers, brokers and their advisors, who are responsible for driving the market forward through th

In [58]:
intelligent_dict=intelligent


intelligent_df= pd.DataFrame(columns=['scrappedUrl','eventname','startdate','enddate','timing','eventinfo','ticketlist','orgProfile','orgName','orgWeb','logo','sponsor','agendalist','type','category','city','country','venue','event_website','googlePlaceUrl','ContactMail','Speakerlist','online_event'],data=intelligent_dict)
intelligent_df.to_csv("intelligent.tsv", sep = '\t',index=False)
print('Dusted')

Dusted


In [ ]:
# driver.switch_to.window(driver.window_handles[0])

In [ ]:
# s=1
# for k in cards:
#     #time.sleep(1)
#     driver.execute_script("window.open('');")
#     driver.switch_to.window(driver.window_handles[s])
#     driver.get(k[0])
#     s+=1
# print(s+1)

In [ ]:
# for k in range(1,31):
#     time.sleep(1)
#     driver.switch_to.window(driver.window_handles[k])
